In [7]:
client_id=''
client_secret = ''
access_token = ''

In [8]:
import requests

url = 'https://www.strava.com/api/v3/athlete'
headers = {
    'Authorization': 'Bearer '+access_token
}

response = requests.get(url, headers=headers)

# Per stampare lo status code della risposta
print("Status Code:", response.status_code)

# Per stampare il contenuto della risposta (ad esempio, in formato JSON)
print("Response Content:", response.json())


Status Code: 200
Response Content: {'id': 135306459, 'username': None, 'resource_state': 2, 'firstname': 'prova', 'lastname': 'test', 'bio': None, 'city': 'Selvazzano Dentro', 'state': 'Veneto', 'country': 'Italy', 'sex': None, 'premium': False, 'summit': False, 'created_at': '2024-04-15T17:59:08Z', 'updated_at': '2024-04-15T18:04:52Z', 'badge_type_id': 0, 'weight': None, 'profile_medium': 'https://d3nn82uaxijpm6.cloudfront.net/assets/avatar/athlete/medium-bee27e393b8559be0995b6573bcfde897d6af934dac8f392a6229295290e16dd.png', 'profile': 'https://d3nn82uaxijpm6.cloudfront.net/assets/avatar/athlete/large-800a7033cc92b2a5548399e26b1ef42414dd1a9cb13b99454222d38d58fd28ef.png', 'friend': None, 'follower': None}


In [9]:
from IPython.display import display, Markdown

# Supponiamo che questa sia la variabile dove hai salvato l'access_token

markdown_text = f"""
Per ottenere le informazioni sull'atleta, copia e incolla il seguente url nel tuo Browser, autorizza le api alla lettura e poi ricordati di inserire il nuovo url della pagina web nella cella sottostante:

```bash
https://www.strava.com/oauth/authorize?client_id={client_id}&redirect_uri=http://localhost&response_type=code&scope=activity:read_all"""

display(Markdown(markdown_text))



Per ottenere le informazioni sull'atleta, copia e incolla il seguente url nel tuo Browser, autorizza le api alla lettura e poi ricordati di inserire il nuovo url della pagina web nella cella sottostante:

```bash
https://www.strava.com/oauth/authorize?client_id=124982&redirect_uri=http://localhost&response_type=code&scope=activity:read_all

In [10]:
url_restituito='http://localhost/?state=&code=48b09c053d5d63bffd50adc0fb9f9e796447a200&scope=read,activity:read_all'

In [11]:
from urllib.parse import urlparse, parse_qs

# L'URL da cui estrarre il codice
url = url_restituito
# Analizza l'URL per ottenere i componenti
parsed_url = urlparse(url)

# Estrai la stringa di query dall'URL analizzato
query_string = parsed_url.query

# Analizza la stringa di query per ottenere un dizionario dei parametri
query_params = parse_qs(query_string)

# Ora puoi ottenere il valore del codice
code = query_params["code"][0]  # '55f45adede3de7b8335cfc46ec4e8199d5bf7b62'
print(code)


48b09c053d5d63bffd50adc0fb9f9e796447a200


In [12]:
import requests

# Sostituisci questi valori con i tuoi dati reali
client_id = client_id
client_secret = client_secret
code = code  # Il codice di autorizzazione ottenuto

url = 'https://www.strava.com/oauth/token'

data = {
    'client_id': client_id,
    'client_secret': client_secret,
    'code': code,
    'grant_type': 'authorization_code'
}

response = requests.post(url, data=data)

if response.status_code == 200:
    # La richiesta è andata a buon fine; estrai il token di accesso dalla risposta
    tokens = response.json()
    access_token = tokens['access_token']
    refresh_token = tokens['refresh_token']  # Puoi salvarlo per rinnovare l'accesso in futuro
    print("Access Token:", access_token, refresh_token)
    # Puoi anche stampare o salvare il refresh_token se necessario
else:
    print("Errore nella richiesta:", response.status_code, response.text)


Access Token: 745fb8ecdc864ebc0e88785645181d4b54460ded f027c2b8f4198ad407d5b8fa0cbff2ae5ad65121


# DA QUI IN POI RIPETERE PER SCARICARE I NUOVI DATI, GLI STEP PRECEDENTI NON SONO DA RIPETERE.

In [18]:
import pandas as pd
import requests


auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"

payload = {
    'client_id':    client_id,
    'client_secret': client_secret,
    'refresh_token': refresh_token,
    'grant_type': "refresh_token",
    'f': 'json'
}

print("Requesting Token...\n")
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']
print("Access Token = {}\n".format(access_token))
datasets = []
page =1 
page_not_empty = True
while page_not_empty:
    header = {'Authorization': 'Bearer ' + access_token}
    param = {'per_page': 200, 'page': page}
    response = requests.get(activites_url, headers=header, params=param)
    
    if response.status_code != 200:
        print("Failed to retrieve data for page {}".format(page))
        break

    my_dataset = pd.DataFrame(response.json())
    
    if my_dataset.empty:
        break

    datasets.append(my_dataset)
    print("Retrieved page {}".format(page))
    page += 1

    activities_df = pd.concat(datasets, ignore_index=True)
else:
    print("Failed to retrieve access token")






Requesting Token...



/Users/mircomiglietta/Documents/RUNTRAKER/codice/my_env/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.strava.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Access Token = 745fb8ecdc864ebc0e88785645181d4b54460ded

Retrieved page 1


## Stampo i nomi delle colonne del Df per capire quali tenere 

In [19]:
print(activities_df)

   resource_state                                 athlete          name  \
0               2  {'id': 135306459, 'resource_state': 1}  Corsa serale   

   distance  moving_time  elapsed_time  total_elevation_gain type sport_type  \
0   10000.0         3600          3600                     0  Run        Run   

   workout_type  ...  max_speed has_heartrate heartrate_opt_out  \
0             3  ...          0         False             False   

  display_hide_heartrate_option  upload_id external_id from_accepted_tag  \
0                         False       None        None             False   

  pr_count  total_photo_count  has_kudoed  
0        0                  0       False  

[1 rows x 44 columns]


In [17]:
activities_df.to_csv('RunTracker.csv', sep=';')

## Stampo il df con solo le colonne di mio interesse

In [21]:
columns_relevant = ['average_heartrate'
,'average_speed'
,'name'
,'distance'
,'has_heartrate'
,'max_speed'
,'moving_time'
,'sport_type'
,'start_date']

activities_df[columns_relevant]


KeyError: "['average_heartrate'] not in index"

## Calcolo la velocità in km/h e minuti al kilometro

In [4]:
activities_df['average_speed_min_km'] =((activities_df['moving_time']/60)/(activities_df['distance']/1000)).astype(int)+(((activities_df['moving_time']/60)/(activities_df['distance']/1000)%1)*0.6)
activities_df['average_speed_km_h'] =((activities_df['distance']/1000)/(activities_df['moving_time']/3600))


## Creo una somma progressia della colonna distanza partendo dalla corsa meno recente fino alla più recente

In [5]:
# Convertire la colonna 'Data' in un formato di data
activities_df['start_date'] = pd.to_datetime(activities_df['start_date'])

# Ordinare il DataFrame in base alla colonna 'Data'
activities_df = activities_df.sort_values(by='start_date', ascending=True)

# Calcolare la somma cumulativa della colonna 'Valore'
activities_df['cumulative_km'] = (activities_df['distance']/1000).cumsum()

## Stampo il dataframe con l'aggiunta delle colonne calcolate

In [6]:
column_relevant = ['average_heartrate'
,'average_speed'
,'average_speed_min_km'
,'average_speed_km_h'
,'name'
,'distance'
,'cumulative_km'
,'max_heartrate'
,'max_speed'
,'moving_time'
,'sport_type'
,'start_date']

activities_df[column_relevant]


,average_heartrate,average_speed,average_speed_min_km,average_speed_km_h,name,distance,cumulative_km,max_heartrate,max_speed,moving_time,sport_type,start_date
140,166.5,2.190,7.366838,7.882916,Corsa pomeridiana,6398.3,6.3983,180.0,4.026,2922,Run,2022-11-04 12:01:22+00:00
139,161.7,2.146,7.459205,7.726641,Corsa dell'ora di pranzo,7692.3,14.0906,178.0,3.997,3584,Run,2022-11-05 10:38:55+00:00
138,166.7,2.283,7.180646,8.217966,Corsa mattutina,7339.1,21.4297,184.0,3.938,3215,Run,2022-11-07 05:38:44+00:00
137,170.2,2.412,6.545808,8.683471,Corsa pomeridiana,21014.0,42.4437,182.0,4.094,8712,Run,2022-11-13 15:13:42+00:00
136,168.8,2.228,7.288954,8.019685,Corsa mattutina,5092.5,47.5362,183.0,4.098,2286,Run,2022-11-18 05:07:26+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
4,142.1,2.337,7.078602,8.413964,Corsa mattutina,15023.6,1373.3834,166.0,4.124,6428,Run,2024-04-08 03:38:47+00:00
3,138.9,2.186,7.374315,7.870031,Corsa mattutina,7008.7,1380.3921,147.0,3.090,3206,Run,2024-04-09 03:51:47+00:00
2,163.4,2.816,5.550825,10.138490,Corsa mattutina,8614.9,1389.0070,178.0,4.034,3059,Run,2024-04-10 04:09:50+00:00
1,155.3,2.488,6.418504,8.958558,Corsa mattutina,9598.1,1398.6051,168.0,3.842,3857,Run,2024-04-12 03:52:18+00:00


In [65]:
activities_df.to_csv('RunTracker.csv', sep=';')